In [1]:
# Working: Installation of dependencies
# Working: Cloned the SciGen Repo
# WIP: 

In [2]:
# Install the requirements
!pip uninstall -y torch
!pip install torch==1.5.1
!pip uninstall -y transformers
!pip install transformers==2.10.0

!pip install pytorch-lightning==0.8.1
!pip install bert-score==0.3.3
!pip install sacrebleu==1.4.12
!pip install moverscore==1.0.3

# For meteor_score
!pip install -U nltk

# For BLEURT
!git clone https://github.com/google-research/bleurt.git
!cd bleurt; pip install .

#! pip install parsimonious==0.8.1

Found existing installation: torch 1.5.1
Uninstalling torch-1.5.1:
  Successfully uninstalled torch-1.5.1
  Using cached torch-1.5.1-cp37-cp37m-manylinux1_x86_64.whl (753.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.5.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.1 which is incompatible.
Found existing installation: transformers 2.10.0
Uninstalling transformers-2.10.0:
  Successfully uninstalled transformers-2.10.0
  Using cached transformers-2.10.0-py3-none-any.whl (660 kB)
fatal: destination path 'bleurt' already exists and is not an empty directory.
Processing /content/bleurt
  DEPRECATION: A future pip version will change local packages to be 

In [3]:
import transformers
print('Transformers:', transformers.__version__)
import torch
print('Torch:', torch.__version__)
print('Cuda available: ', torch.cuda.is_available())

Transformers: 2.10.0
Torch: 1.5.1
Cuda available:  True


In [4]:
# Clone the SciGen repo
!cd ..
!git clone https://github.com/DarshanAdiga/SciGen.git
!cd SciGen; git checkout feature/reproducing;

Cloning into 'SciGen'...
remote: Enumerating objects: 484, done.
remote: Counting objects: 100% (484/484), done.
remote: Compressing objects: 100% (463/463), done.
remote: Total 484 (delta 20), reused 458 (delta 12), pack-reused 0
Receiving objects: 100% (484/484), 39.76 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Checking out files: 100% (429/429), done.
Branch 'feature/reproducing' set up to track remote branch 'feature/reproducing' from 'origin'.
Switched to a new branch 'feature/reproducing'


In [5]:
cd SciGen/

/content/SciGen


In [6]:
# from ..baselines import *
# FIXME This method of importing is a hacky way. Avoid doing this
import sys
import os
sys.path.append(os.path.abspath('baselines'))
import callbacks, convert_json_files, evaluate, lightning_base, train_table2text_bart, train_table2text_t5, utils

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

## Setups

In [7]:
# Extract train/large/train.zip into json file; The option -j prevents creation of subdirectories while extracting
!cd ./dataset/train/large/; unzip -j train.zip

Archive:  train.zip
  inflating: train.json              


In [8]:
## Optionally
# !rm -rf ./reproducing/processed_data

In [9]:
# Create the directory structure to store the converted train,test & development datasets
processed_data_dir = 'reproducing/processed_data/'
if not os.path.exists(processed_data_dir):
  # Create a base directory
  os.makedirs(processed_data_dir)
  # Create directories for each experiment
  for part in ['few-shot', 'large', 'medium']:
    os.makedirs(processed_data_dir + part)
  # Create test dir structure
  os.makedirs(processed_data_dir + 'test/')
  print(f'Created the necessary directory structures under {processed_data_dir}')

Created the necessary directory structures under reproducing/processed_data/


## Running the SciGen Baseline results

### Preprocessing of all the files one-b-one

In [10]:
# Development
! export SPLIT="few-shot"; export PART="development/$SPLIT/dev"; python ./baselines/convert_json_files.py -f ./dataset/$PART".json" -s ./reproducing/processed_data/$SPLIT/dev
! export SPLIT="large"; export PART="development/$SPLIT/dev"; python ./baselines/convert_json_files.py -f ./dataset/$PART".json" -s ./reproducing/processed_data/$SPLIT/dev
! export SPLIT="medium"; export PART="development/$SPLIT/dev"; python ./baselines/convert_json_files.py -f ./dataset/$PART".json" -s ./reproducing/processed_data/$SPLIT/dev

In [11]:
# Test
! export PART='test/test-CL'; python ./baselines/convert_json_files.py -f ./dataset/$PART".json" -s ./reproducing/processed_data/$PART
! export PART='test/test-Other'; python ./baselines/convert_json_files.py -f ./dataset/$PART".json" -s ./reproducing/processed_data/$PART

In [12]:
# Train
! export SPLIT="few-shot"; export PART="train/$SPLIT/train"; python ./baselines/convert_json_files.py -f ./dataset/$PART".json" -s ./reproducing/processed_data/$SPLIT/train
! export SPLIT="large"; export PART="train/$SPLIT/train"; python ./baselines/convert_json_files.py -f ./dataset/$PART".json" -s ./reproducing/processed_data/$SPLIT/train
! export SPLIT="medium"; export PART="train/$SPLIT/train"; python ./baselines/convert_json_files.py -f ./dataset/$PART".json" -s ./reproducing/processed_data/$SPLIT/train

## Run the BART Finetuning

### 1. Experiment on 'few-shot' data

In [24]:
# Set the input split and output directories for a given experiment
%env SPLIT=few-shot
%env OUTPUT_DIR=./reproducing/fewshot_output

# Remove the output directory, if exists
! rm -r $OUTPUT_DIR

env: SPLIT=few-shot
env: OUTPUT_DIR=./reproducing/fewshot_output
rm: cannot remove './reproducing/fewshot_output': No such file or directory


In [25]:
! python ./baselines/train_table2text_bart.py \
--data_dir=./reproducing/processed_data/$SPLIT \
--model_name_or_path=facebook/bart-large \
--learning_rate=3e-5 \
--num_train_epochs 30 \
--train_batch_size=8 \
--eval_batch_size=4 \
--test_batch_size=4 \
--output_dir=$OUTPUT_DIR \
--n_gpu 1 \
--do_train \
--do_predict \
--early_stopping_patience 10 \
--max_source_length 384 \
--max_target_length 384 \

2022-02-07 22:31:51 connectionpool - _new_conn: Starting new HTTPS connection (1): s3.amazonaws.com:443
2022-02-07 22:31:51 connectionpool - _make_request: https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/distilbert-base-uncased-config.json HTTP/1.1" 200 0
2022-02-07 22:31:51 configuration_utils - get_config_dict: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
2022-02-07 22:31:51 configuration_utils - from_dict: Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "outp